<a href="https://colab.research.google.com/github/Yoshiki0418/Deep_Learning/blob/main/Autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AutoGrad

Pytorchでは計算グラフを自動的に構築し、勾配を逆方向に計算することができる。tensor生成時に、`requires_grad=True`を設定することで自動微分を有効にすることができる。計算後に、`.backward`を呼ぶことで自動的に勾配が計算される。

勾配情報は、`.grad`属性に累積されるが、末端ノードに対する勾配しか保存されない。中間ノードに対する勾配を保存したい場合は、当該tensorに対して、`.retain_grad()`を実行する必要がある。

#### PythonでAutoGradを試してみる

今回は以下の式の勾配をAutoGradを使って求める。

<br>

$$
初期のtensorの値：x=2,  y=3
$$

$$
z = y×log(x) + sin(y)
$$

$$
\frac{∂z}{∂a}　及び　\frac{∂z}{∂y}を求める
$$

In [11]:
import torch
from torch import tensor

x = tensor(2. , requires_grad=True)
y = tensor(3. , requires_grad=True)

z = y * torch.log(x) + torch.sin(y)

print(z)
z.backward()

tensor(2.2206, grad_fn=<AddBackward0>)


In [12]:
x.grad

tensor(1.5000)

In [13]:
y.grad

tensor(-0.2968)

In [17]:
# 累積されていくことをもう一度実行することによって確認する
z = y * torch.log(x) + torch.sin(y)
z.backward()

In [18]:
x.grad

tensor(4.5000)

In [19]:
y.grad

tensor(-0.8905)

#### AutoGradで勾配を計算する(中間ノードの勾配)

勾配情報は、末端ノードのみが累積によって保存されていくことを学んだ。しかし、中間ノードを確認したい場合もある。そこで、中間ノードに対する勾配を保存したい場合は、当該tensorに対して、.retain_grad()を実行する。

* 以下の式の勾配をAutogradで求める
$$
初期のtensorの値：x=2,  y=3
$$

$$
z = (x + y)^2
$$

$$
中間ノードとなる\frac{∂z}{∂(x + y)}を求める
$$

In [22]:
x = tensor(2. , requires_grad=True)
y = tensor(3. , requires_grad=True)

# u を x と y の合計として定義し、後で勾配を取得するために勾配保持を有効にする
u = x + y
u.retain_grad()

z = u ** 2

print(z)
z.backward()

tensor(25., grad_fn=<PowBackward0>)


In [23]:
u.grad

tensor(10.)

In [25]:
x.grad

tensor(10.)

In [26]:
y.grad

tensor(10.)

#### 勾配を保持する必要がない場合

Autogradで勾配を保持するには、計算グラフを構築するため、計算量が高くなりメモリ使用量も増える。したがって、勾配を計算する必要がないケースでは、`with torch.no_grad():`を使って勾配を計算しないようにすることで、計算速度が向上し、メモリ使用量が減少する。

<br>

では、どのような場面で使用することがあるのか？

→モデルの推論(予測)時やパラメータの更新時に使用する。

In [28]:
x = tensor(2. , requires_grad=True)
y = tensor(3. , requires_grad=True)

with torch.no_grad():
  z = y * torch.log(x) + torch.sin(y)
  print(z)

t = y * torch.log(x) + torch.sin(y)
print(t)

# z.backward(). => torch.no_grad():内で計算しているので、勾配が計算されていないので使用することができない
t.backward()

tensor(2.2206)
tensor(2.2206, grad_fn=<AddBackward0>)
